# AI model


In [2]:
import torch 

## read tensors

In [5]:
data_path = '../data/tensors/'

# Load the preprocessed and saved data
X_train_tensor, y_train_tensor = torch.load(data_path + 'train_data.pt')
X_test_tensor, y_test_tensor = torch.load(data_path + 'test_data.pt')

# Load normalization parameters (mean, std)
mean, std = torch.load(data_path + 'normalization_params.pt')

# You can also check the loaded data if you want
print(f"Loaded training data: X_train shape: {X_train_tensor.shape}, y_train shape: {y_train_tensor.shape}")
print(f"Loaded test data: X_test shape: {X_test_tensor.shape}, y_test shape: {y_test_tensor.shape}")


Loaded training data: X_train shape: torch.Size([4257, 64]), y_train shape: torch.Size([4257, 1])
Loaded test data: X_test shape: torch.Size([1065, 64]), y_test shape: torch.Size([1065, 1])


C:\Users\reza\AppData\Local\Temp\ipykernel_37232\560550039.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train_tensor, y_train_tensor = torch.load(data_path + 'train

## base model

In [9]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class MLPmodelSigmoidhead(nn.Module):
    def __init__(self, input_size):
        super(MLPmodelSigmoidhead, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.output(x))  # Output probability between 0 and 1
        return x


## train loop

In [17]:
# Initialize the model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
model = MLPmodelSigmoidhead(input_size=input_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader for training and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
criterion.to(device)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()  # Zero gradients

        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * X_batch.size(0)
    
    # Average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

model_output ='../models/'
# Save the model after training if needed
# Save the model and other useful info
torch.save({
    'model_state_dict': model.state_dict(),
    'input_size': input_size,
    'mean': mean,  # Optional: save normalization parameters
    'std': std,
}, model_output + 'model_checkpoint.pth')
print("Model saved successfully.")


Epoch [1/50], Loss: 0.6190
Epoch [2/50], Loss: 0.5824
Epoch [3/50], Loss: 0.5632
Epoch [4/50], Loss: 0.5504
Epoch [5/50], Loss: 0.5461
Epoch [6/50], Loss: 0.5424
Epoch [7/50], Loss: 0.5458
Epoch [8/50], Loss: 0.5438
Epoch [9/50], Loss: 0.5430
Epoch [10/50], Loss: 0.5407
Epoch [11/50], Loss: 0.5409
Epoch [12/50], Loss: 0.5444
Epoch [13/50], Loss: 0.5395
Epoch [14/50], Loss: 0.5400
Epoch [15/50], Loss: 0.5393
Epoch [16/50], Loss: 0.5439
Epoch [17/50], Loss: 0.5392
Epoch [18/50], Loss: 0.5438
Epoch [19/50], Loss: 0.5397
Epoch [20/50], Loss: 0.5392
Epoch [21/50], Loss: 0.5377
Epoch [22/50], Loss: 0.5376
Epoch [23/50], Loss: 0.5393
Epoch [24/50], Loss: 0.5443
Epoch [25/50], Loss: 0.5446
Epoch [26/50], Loss: 0.5431
Epoch [27/50], Loss: 0.5463
Epoch [28/50], Loss: 0.5413
Epoch [29/50], Loss: 0.5500
Epoch [30/50], Loss: 0.5460
Epoch [31/50], Loss: 0.5454
Epoch [32/50], Loss: 0.5439
Epoch [33/50], Loss: 0.5445
Epoch [34/50], Loss: 0.5437
Epoch [35/50], Loss: 0.5445
Epoch [36/50], Loss: 0.5475
E